In [1]:
import pandas as pd
import numpy as np
import collections
import re
import sqlite3

In [2]:
# Change this to your path to the DB file!
database = r"C:\Users\14029\Desktop\coaching\coaching\flights\flights.db"
conn = sqlite3.connect(database)

### Load raw data 

In [3]:
# load our datasets
# These first four are used as lookup tables
airlines =     pd.read_sql('SELECT * FROM airlines', con=conn)
airports =     pd.read_sql('SELECT * FROM airports', con=conn)
cancel_codes = pd.read_sql('SELECT * FROM cancel_codes', con=conn)
carriers =     pd.read_sql('SELECT * FROM carriers', con=conn)
# This has all the data about flights
flights =      pd.read_sql('SELECT * FROM flights', con=conn)

In [4]:
# load our datasets
# These first four are used as lookup tables
airlines =     pd.read_sql('SELECT * FROM airlines', con=conn)
airports =     pd.read_sql('SELECT * FROM airports', con=conn)
cancel_codes = pd.read_sql('SELECT * FROM cancel_codes', con=conn)
carriers =     pd.read_sql('SELECT * FROM carriers', con=conn)
# This has all the data about flights
flights =      pd.read_sql('SELECT * FROM flights', con=conn)

In [5]:
#Closing the connection to the db, since we're done using it
conn.close()

# Pandas 1

Challenge 1 - Describing the flights data frame

In [14]:
flights.shape

(410517, 32)

In [15]:
#First 5 rows
flights.head(5)

,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST_AIRPORT_ID,...,DIVERTED,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 31
0,1,3,2017-02-01,B6,20409,33,10785,"Burlington, VT",Vermont,12478,...,0,92.0,1,266,NaN,NaN,NaN,NaN,NaN,None
1,1,3,2017-02-01,B6,20409,34,12478,"New York, NY",New York,10785,...,0,78.0,1,266,NaN,NaN,NaN,NaN,NaN,None
2,1,3,2017-02-01,B6,20409,42,12954,"Long Beach, CA",California,14570,...,0,81.0,1,402,NaN,NaN,NaN,NaN,NaN,None
3,1,3,2017-02-01,B6,20409,43,14570,"Reno, NV",Nevada,12954,...,0,74.0,1,402,NaN,NaN,NaN,NaN,NaN,None
4,1,3,2017-02-01,B6,20409,49,10721,"Boston, MA",Massachusetts,14685,...,0,164.0,1,901,NaN,NaN,NaN,NaN,NaN,None


In [16]:
#data types of df
flights.dtypes

DAY_OF_MONTH             int64
DAY_OF_WEEK              int64
FL_DATE                 object
UNIQUE_CARRIER          object
AIRLINE_ID               int64
FL_NUM                   int64
ORIGIN_AIRPORT_ID        int64
ORIGIN_CITY_NAME        object
ORIGIN_STATE_NM         object
DEST_AIRPORT_ID          int64
DEST_CITY_NAME          object
DEST_STATE_NM           object
DEP_TIME               float64
DEP_DELAY              float64
TAXI_OUT               float64
WHEELS_OFF             float64
WHEELS_ON              float64
TAXI_IN                float64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED                int64
CANCELLATION_CODE       object
DIVERTED                 int64
ACTUAL_ELAPSED_TIME    float64
FLIGHTS                  int64
DISTANCE                 int64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed: 31             object
dtype: o

In [17]:
#show list of columns
flights.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'UNIQUE_CARRIER',
       'AIRLINE_ID', 'FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_NM', 'DEST_AIRPORT_ID', 'DEST_CITY_NAME', 'DEST_STATE_NM',
       'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON',
       'TAXI_IN', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'CANCELLATION_CODE',
       'DIVERTED', 'ACTUAL_ELAPSED_TIME', 'FLIGHTS', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY', 'Unnamed: 31'],
      dtype='object')

In [21]:
# statistical breakdown of the numerical variables
flights.describe()

,DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CANCELLED,DIVERTED,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
count,410517.000000,410517.000000,410517.000000,410517.000000,410517.000000,410517.000000,404337.000000,404335.000000,404255.000000,404255.000000,...,410517.000000,410517.000000,403225.000000,410517.0,410517.000000,64264.000000,64264.000000,64264.000000,64264.000000,64264.000000
mean,14.639238,3.900823,19901.976286,2079.251992,12695.619597,12695.554593,1329.913097,7.082793,16.316048,1353.039386,...,0.015376,0.002387,141.123405,1.0,847.026576,18.930677,2.577602,15.938597,0.042886,22.681019
std,8.099569,1.979376,381.966811,1725.704840,1532.511319,1532.386746,490.151341,38.878867,8.832381,490.432768,...,0.123042,0.048801,76.461818,0.0,614.252176,59.800184,22.843919,32.303749,1.704423,45.299837
min,1.000000,1.000000,19393.000000,1.000000,10135.000000,10135.000000,1.000000,-59.000000,1.000000,1.000000,...,0.000000,0.000000,16.000000,1.0,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,2.000000,19790.000000,692.000000,11292.000000,11292.000000,920.000000,-6.000000,11.000000,935.000000,...,0.000000,0.000000,85.000000,1.0,391.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,4.000000,19805.000000,1599.000000,12892.000000,12892.000000,1326.000000,-3.000000,14.000000,1339.000000,...,0.000000,0.000000,123.000000,1.0,680.000000,0.000000,0.000000,4.000000,0.000000,0.000000
75%,22.000000,6.000000,20304.000000,2981.000000,14057.000000,14057.000000,1734.000000,4.000000,19.000000,1747.000000,...,0.000000,0.000000,174.000000,1.0,1089.000000,16.000000,0.000000,20.000000,0.000000,27.000000
max,28.000000,7.000000,21171.000000,7439.000000,16218.000000,16218.000000,2400.000000,1668.000000,175.000000,2400.000000,...,1.000000,1.000000,745.000000,1.0,4983.000000,1440.000000,1175.000000,1099.000000,332.000000,1306.000000


Challenge 2: Code/answer the following using pandas

In [39]:
# Filter the 'flights' df to only flights landed in 'New York' (desitination)
# has 14107 rows
NY = flights.loc[flights['DEST_CITY_NAME'] == 'New York, NY']
NY

,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST_AIRPORT_ID,...,DIVERTED,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 31
0,1,3,2017-02-01,B6,20409,33,10785,"Burlington, VT",Vermont,12478,...,0,92.0,1,266,NaN,NaN,NaN,NaN,NaN,None
8,1,3,2017-02-01,B6,20409,54,14027,"West Palm Beach/Palm Beach, FL",Florida,12478,...,0,142.0,1,1028,NaN,NaN,NaN,NaN,NaN,None
12,1,3,2017-02-01,B6,20409,62,14027,"West Palm Beach/Palm Beach, FL",Florida,12953,...,0,167.0,1,1035,7.0,0.0,9.0,0.0,8.0,None
15,1,3,2017-02-01,B6,20409,72,14869,"Salt Lake City, UT",Utah,12478,...,0,219.0,1,1990,NaN,NaN,NaN,NaN,NaN,None
16,1,3,2017-02-01,B6,20409,76,13495,"New Orleans, LA",Louisiana,12478,...,0,157.0,1,1182,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410438,28,2,2017-02-28,B6,20409,2301,10792,"Buffalo, NY",New York,12478,...,0,68.0,1,301,NaN,NaN,NaN,NaN,NaN,None
410443,28,2,2017-02-28,B6,20409,2401,10792,"Buffalo, NY",New York,12478,...,0,69.0,1,301,NaN,NaN,NaN,NaN,NaN,None
410457,28,2,2017-02-28,B6,20409,2701,10792,"Buffalo, NY",New York,12478,...,0,NaN,1,301,NaN,NaN,NaN,NaN,NaN,None
410461,28,2,2017-02-28,B6,20409,2807,14321,"Portland, ME",Maine,12478,...,0,99.0,1,273,NaN,NaN,NaN,NaN,NaN,None


In [42]:
#What was the average arrival time
NY['ARR_TIME'].mean()

1485.8989696834926

Challenge 3: Make a list of the unique carriers using pandas methods

In [47]:
unique = flights['UNIQUE_CARRIER']

0         B6
1         B6
2         B6
3         B6
4         B6
          ..
410512    HA
410513    HA
410514    HA
410515    HA
410516    HA
Name: UNIQUE_CARRIER, Length: 410517, dtype: object

In [52]:
unique = carriers.Description.unique()

In [53]:
unique

array(['Titan Airways', 'Tradewind Aviation', 'Comlux Aviation, AG', ...,
       'Airbc Ltd.', 'Atlantic Gulf Airlines', 'Skyway Aviation Inc.'],
      dtype=object)